In [61]:
import yfinance as yf
import pandas as pd
import openpyxl

# Reusable functions

In [62]:
def calculate_return(df):
    if df.empty:
        return None
    # Get the closing price at the start of the year
    start_price = df['Close'].iloc[0]
    
    # Get the closing price at the end of the year
    end_price = df['Close'].iloc[-1]
    
    # Calculate the one-year return
    return (end_price - start_price) / start_price

In [63]:
largest_market_cap = pd.read_excel('./Top Companies by Market Cap.xlsx', sheet_name='Sheet1')
largest_market_cap

,Symbol,Company Name,Industry,Market Cap
0,AAPL,Apple Inc.,Consumer Electronics,"2,811.03B"
1,MSFT,Microsoft Corporation,Software - Infrastructure,"2,469.80B"
2,GOOGL,Alphabet Inc.,Internet Content & Information,"1,779.01B"
3,GOOG,Alphabet Inc.,Internet Content & Information,"1,778.79B"
4,AMZN,"Amazon.com, Inc.",Internet Retail,"1,360.20B"
...,...,...,...,...
495,RBLX,Roblox Corporation,Electronic Gaming & Multimedia,19.59B
496,WBA,"Walgreens Boots Alliance, Inc.",Pharmaceutical Retailers,19.51B
497,TW,Tradeweb Markets Inc.,Capital Markets,19.51B
498,ZM,"Zoom Video Communications, Inc.",Software - Application,19.51B


In [64]:
largest_market_cap = largest_market_cap[~largest_market_cap['Symbol'].str.contains('\.')].reset_index(drop=True)
top_300 = largest_market_cap.head(100)
top_300_symbols = top_300['Symbol']

In [65]:
# ticker = yf.Ticker('AAPL')
# stock_info = ticker.info
# print(stock_info.get('beta'))

In [66]:
one_year_data = {}
three_year_data = {}
five_year_data = {} 
for symbol in top_300_symbols:
    ticker = yf.Ticker(symbol)
    one_year_data[symbol] = ticker.history(period="1y")
    three_year_data[symbol] = ticker.history(period="3y")
    five_year_data[symbol] = ticker.history(period="5y")

In [67]:
one_year_return_data = {}
three_year_return_data = {}
five_year_return_data = {}
for key, df in one_year_data.items():
    one_year_return_data[key] = calculate_return(one_year_data[key])
    three_year_return_data[key] = calculate_return(three_year_data[key])
    five_year_return_data[key] = calculate_return(five_year_data[key])

In [68]:
filtered_data_one_year = {key: value for key, value in one_year_return_data.items() if value is not None and value >= 0.15 and value <= 0.8}
filtered_data_three_year = {key: value for key, value in three_year_return_data.items() if value is not None and value >= 0.10}
filtered_data_five_year = {key: value for key, value in five_year_return_data.items() if value is not None and value >= 0.10}

print(filtered_data_one_year.values())
print(filtered_data_three_year.values())
print(filtered_data_five_year.values())
overlapping_keys = set(filtered_data_one_year.keys()) & set(filtered_data_three_year.keys()) & set(filtered_data_five_year .keys())

dict_values([0.3001229342405525, 0.447742977442413, 0.42253525355569616, 0.4260135974055348, 0.21412527065972564, 0.22503531093159448, 0.31191968959202226, 0.44408532092722114, 0.24416840915969312, 0.37073155358596205, 0.3871576335724418, 0.18707175944906795, 0.7088244851141176, 0.15919878141608512, 0.2561649537803035, 0.324882498450057, 0.5983983327461118, 0.15064366619694308, 0.3855724224424731, 0.3785622191440767, 0.2507871208190861, 0.3426759063170163, 0.43854587668297834, 0.213300844077826, 0.7289288498376517, 0.3919855750693312, 0.5007318409553644, 0.5464347507642663, 0.6281536400248436, 0.3945395517853608, 0.5474634570349849, 0.43513807731660054, 0.41258623966357333, 0.16838748430543027, 0.1555847347509657, 0.2045448678116866, 0.5367854269777068, 0.42952280577328844, 0.2791187805923465, 0.21143150165271393, 0.2815058222899161, 0.3887346011135765, 0.6083215046765759, 0.3772141576884442, 0.16231465792444863, 0.7659968604248233, 0.19407966991261863, 0.3257044539246005])
dict_values

In [69]:
len(overlapping_keys)

34

In [70]:
betas = {}
for key in overlapping_keys:
    betas[key] = yf.Ticker(key).info.get('beta')
filtered_beta = {key: value for key, value in betas.items() if value is not None and value > 0.70 and value <= 1.50}
print(len(filtered_beta))


19


In [71]:
df_filtered_beta_keys = pd.DataFrame(list(filtered_beta.keys()), columns=['Symbol'])
df_filtered_beta_keys

,Symbol
0,ASML
1,GOOG
2,ACN
3,AAPL
4,SONY
5,INTU
6,V
7,MA
8,BX
9,TTE


In [72]:
merged_df = pd.merge(df_filtered_beta_keys, largest_market_cap, on='Symbol', how='left')
merged_df

,Symbol,Company Name,Industry,Market Cap
0,ASML,ASML Holding N.V.,Semiconductor Equipment & Materials,238.48B
1,GOOG,Alphabet Inc.,Internet Content & Information,"1,778.79B"
2,ACN,Accenture plc,Information Technology Services,196.59B
3,AAPL,Apple Inc.,Consumer Electronics,"2,811.03B"
4,SONY,Sony Group Corporation,Consumer Electronics,105.39B
5,INTU,Intuit Inc.,Software - Application,151.78B
6,V,Visa Inc.,Credit Services,490.17B
7,MA,Mastercard Incorporated,Credit Services,373.76B
8,BX,Blackstone Inc.,Asset Management,130.52B
9,TTE,TotalEnergies SE,Oil & Gas Integrated,156.18B


### Semiconductor Equipment & Materials: ASML Holding N.V. (ASML)
### Internet Content & Information: Alphabet Inc. (GOOG or GOOGL)
### Information Technology Services: Accenture plc (ACN)
### Consumer Electronics: Apple Inc. (AAPL)
### Software - Application: Intuit Inc. (INTU)
### Credit Services: Visa Inc. (V)
### Asset Management: Blackstone Inc. (BX)
### Oil & Gas Integrated: TotalEnergies SE (TTE)
### Travel Services: Booking Holdings Inc. (BKNG)
### Software - Infrastructure: Microsoft Corporation (MSFT)